In [1]:
import torch
from torch.multiprocessing import Pool, Array
import numpy as np
import time
import gc

rng = np.random.default_rng()
torch.cuda.device_count()

2

# single GPU memory in R11 computer

can fit 500 samples, 40000 imp wts for each, 10 classes, computes trace in 31 seconds

In [14]:
# def initpool(arr):
#     global sharedArr
#     sharedArr = arr

# def trace_for_chunk(xt_, rank, chunkSize, num_gpus, currentInv, fisher, gpu_id):
#     upper_bound = int(xt_.shape[0]/(num_gpus-gpu_id))
#     lower_bound = int(upper_bound-(xt_.shape[0]/num_gpus))
#     t = time.time()
#     traceEst = np.frombuffer(sharedArr.get_obj(), dtype=np.float64)
#     # print("Beginning GPU ", gpu_id, " at time: ", time.time(), flush=True)
#     for c_idx in range(lower_bound, upper_bound, chunkSize):
#         xt_chunk = xt_[c_idx : c_idx + chunkSize]
#         # xt_chunk = torch.tensor(xt_chunk).clone().detach().cuda(gpu_id)
#         xt_chunk = xt_chunk.clone().detach().cuda(gpu_id)
#         currentInv = currentInv.cuda(gpu_id)
#         fisher = fisher.cuda(gpu_id)
#         innerInv = torch.inverse(torch.eye(rank).cuda(gpu_id) + xt_chunk @ currentInv @ xt_chunk.transpose(1, 2))
#         # print('fisher: ', fisher, '\n', 'curentinv: ', currentInv, '\n', 'xt_chunk: ', xt_chunk)
#         innerInv[torch.where(torch.isinf(innerInv))] = torch.sign(innerInv[torch.where(torch.isinf(innerInv))]) * np.finfo('float32').max
#         traceEst[c_idx : c_idx + chunkSize] = torch.diagonal(
#             xt_chunk @ currentInv @ fisher @ currentInv @ xt_chunk.transpose(1, 2) @ innerInv,
#             dim1=-2,
#             dim2=-1
#         ).sum(-1).detach().cpu()
#         # print(time.time()-t)
#     del xt_chunk, fisher, currentInv
#     # print("Finishing GPU ", gpu_id, " at time: ", time.time(), flush=True)
#     return

from checking_multiproc import initpool, trace_for_chunk

dim = 10 * 3000 #X.shape[-1]
rank = 10 #X.shape[-2]
fisher = torch.rand((dim, dim)).cuda()
currentInv = torch.rand((dim, dim)).cuda()
xt_ = torch.rand((100, rank, dim)).cuda()

## using 2 GPUs
chunkSize = 500
NUM_GPUS = torch.cuda.device_count()
torch.multiprocessing.set_start_method('spawn', force=True)
tE = Array('d', xt_.shape[0], lock=True)

with Pool(processes=NUM_GPUS, initializer=initpool, initargs=(tE,)) as pool:
    args = [(xt_, rank, chunkSize, NUM_GPUS, currentInv, fisher, x) for x in range(NUM_GPUS)]
    result = pool.starmap(trace_for_chunk, args)

traceEst = np.frombuffer(tE.get_obj())

## use 1 GPU
# innerInv = torch.inverse(torch.eye(rank).cuda() + xt_ @ currentInv @ xt_.transpose(1, 2)).detach()
# innerInv[torch.where(torch.isinf(innerInv))] = torch.sign(innerInv[torch.where(torch.isinf(innerInv))]) * np.finfo('float32').max
# traceEst = torch.diagonal(xt_ @ currentInv @ fisher @ currentInv @ xt_.transpose(1, 2) @ innerInv, dim1=-2, dim2=-1).sum(-1)

del fisher, currentInv, xt_, traceEst#, innerInv
torch.cuda.empty_cache()
gc.collect()

0

In [15]:
del fisher 
del currentInv 
del xt_
del tE
del innerInv
del traceEst

NameError: name 'fisher' is not defined

In [19]:
del pool
torch.cuda.empty_cache()
gc.collect()

0

In [21]:
torch.cuda.empty_cache()
gc.collect()

0